In [73]:
# https://github.com/ageitgey/face_recognition
import face_recognition
from PIL import Image, ImageDraw
import os
import pandas as pd
import numpy as np

## Get Image List Function

In [27]:
def get_image_list(path, ignore_list=[None]):
    valid_ext = ('.jpg', '.jpeg')
    list_of_files = os.listdir(path) # Get a list of all files in the current directory
    all_files = []
    # Iterate over all the entries
    for entry in list_of_files:
        # Create full path
        full_path = os.path.join(path, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(full_path) and entry not in ignore_list:
            all_files = all_files + get_image_list(full_path, ignore_list, move, videos)
        else:
            all_files.append(full_path)
                            
    # Return a filtered list that only contains files who end in the valid extension
    return [image for image in all_files if image.lower().endswith(valid_ext)]

## Function to Center all Images in a Given Path

In [182]:
def center_images(path, guides={"left-x": 3039, "right-x": 3217, "bottom-y": 1556}, end_dims=(6016, 4000)):
# def center_images(path, guides={"left-x": 2799, "right-x": 2977, "bottom-y": 2444}, end_dims=(6016, 4000)):
    # Get image list
    image_list = get_image_list(path)
    
    for img in image_list:
        # Get facial features in the image's face
        loaded_image = face_recognition.load_image_file(img)
        face_landmarks = face_recognition.face_landmarks(loaded_image)
        
        # Only center images with one face in it
        if len(face_landmarks) == 1:
            # Get the coordinate list for the left eye
            coords = np.array(face_landmarks[0]['left_eye'])
            x_min, x_max, y_max = np.amin(coords, 0)[0], np.amax(coords, 0)[0], np.amax(coords, 0)[1]
            
            # Transformation values (x, y) -> (ax+by+c, dx+ey+f)
            a = (guides['right-x'] - guides['left-x']) / (x_max - x_min)
            b = 0
            c = guides['left-x'] - (x_min * a)
            d = 0
            e = a
            f = guides['bottom-y'] - (y_max * e)
            
            # Load current image and scale it up so the eye is the proper size
            curr_image = Image.open(img, 'r')
            curr_image = curr_image.transform(curr_image.size, Image.AFFINE, (a, b, c, d, e, f))
            
#             draw = ImageDraw.Draw(curr_image)
#             draw.line([0, y_max*e+f, 6016, y_max*e+f], fill=125, width=5) # Horiz
#             draw.line([x_min*a+c, 0, x_min*a+c, 4000], fill=125, width=5) # Vert1
#             draw.line([x_max*a+c, 0, x_max*a+c, 4000], fill=125, width=5) # Vert2
#             curr_image.save('Outputs/test2.png', 'PNG')
#             return
            
            # Transform and export the image
            new_image = Image.new('RGBA', end_dims, (0, 0, 0, 0))
            new_image.paste(curr_image, (int(c), int(f)))
            new_image.save('Outputs/{}.png'.format(img[len(path):]), format='png')

In [183]:
input_dir = '/Users/CollinHeist/Documents/GitHub/Personal-Projects/Duplicate Picture Identifier/Inputs/'
center_images(input_dir)